In [1]:
import sys
sys.path.insert(0, '../src/')
sys.path.insert(0, '../')

import django
django.setup()

In [2]:
from importlib import reload

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [3]:
import datetime

from apps.processing.ala.models import SamplingFeature, Observation
from apps.processing.ala.util import util

from importlib import import_module
from django.conf import settings
from apps.common.models import Process, Property

import pytz
import time
from psycopg2.extras import DateTimeTZRange
from django.utils.dateparse import parse_datetime
from dateutil.parser import parse
# from datetime import datetime
from functools import partial

# from luminol.anomaly_detector import AnomalyDetector
import luminol.anomaly_detector as lad

from apps.utils.time import UTC_P0100
from apps.common.util.util import generate_intervals
# from apps.ad.anomaly_detection import *
import apps.ad.anomaly_detection as ad

feature = SamplingFeature.objects.get(id_by_provider='11359201')
observed_property = Property.objects.get(name_id='air_temperature')

from apps.mc.api.views import get_observations, parse_date_range, get_empty_slots, get_not_null_ranges, get_feature_nn_from_list

from apps.processing.ala.models import SamplingFeature, Observation
from apps.common.models import Process, Property, TimeSlots
from psycopg2.extras import DateTimeTZRange

In [4]:
topic = "drought"
prop = "air_temperature"
phenomenon_date_from = pd.to_datetime("2019-01-01")
phenomenon_date_to = pd.to_datetime("2019-01-30")
provider_model = Observation
features = provider_model._meta.get_field('feature_of_interest').remote_field.model.objects.all()

def detect_anomalies(
    phenomenon_date_from = phenomenon_date_from,
    phenomenon_date_to = phenomenon_date_to,
    detector_method='bitmap_mod',
    use_baseline=True,
    shift=True,
    extend_range=True,
    detector_params={
        "precision": 6,
        "lag_window_size": 96,
        "future_window_size": 96,
        "chunk_size": 2
    },
    topic = topic,
    prop = prop,
    provider_model = provider_model,
    feature = features[0]
):
    reload(ad)
    reload(lad)

#     item = provider_model._meta.get_field('feature_of_interest').remote_field.model.objects.all()[0]

    observation_provider_name = f"{provider_model.__module__}.{provider_model.__name__}"
    topic_config = settings.APPLICATION_MC.TOPICS.get(topic)

    properties = topic_config['properties']
    ts_id = topic_config['time_slots'][0]
#     ts_config_id = topic_config['time_slots'][0]
#     ts_config = settings.APPLICATION_MC.TIME_SLOTS[ts_config_id]
    prop_config = properties[prop]

    observation_provider_name = f"{Observation.__module__}.{Observation.__name__}"
    process_name_id = prop_config['observation_providers'][observation_provider_name]["process"]
    process = Process.objects.get(name_id=process_name_id)

    prop_item = Property.objects.get(name_id=prop)

    pt_range, day_from, day_to = parse_date_range(str(phenomenon_date_from), str(phenomenon_date_to))

    pt_range_z = DateTimeTZRange(
        pt_range.lower.replace(tzinfo=UTC_P0100),
        pt_range.upper.replace(tzinfo=UTC_P0100)
    )

#     zero = parse_datetime(ts_config['zero'])
#     frequency = ts_config['frequency']
#     range_from = ts_config['range_from']
#     range_to = ts_config['range_to']

#     t = TimeSeries(
#         zero=zero,
#         frequency=frequency,
#         range_from=range_from,
#         range_to=range_to
#     )
    t = TimeSlots.objects.get(name_id=ts_id)
    
    zero = t.zero
#     t.full_clean()
#     t.clean()
    
    nn_feature_ranges = get_not_null_ranges(
#         features=[item],
        features=[feature],
        props=[prop],
        topic_config=topic_config,
        observation_provider_name=observation_provider_name,
        provider_model=provider_model,
        pt_range_z=pt_range_z,
        time_slots=t,
    )
    
    data_range = get_feature_nn_from_list(
        nn_feature_ranges,
#         item,
        feature,
        prop_item.id,
        process.id
    )

    feature_time_slots = get_empty_slots(t, data_range)
    
    get_observations_func = partial(
        get_observations,
        feature_time_slots,
        prop_item,
        provider_model,
#         item,
        feature,
        process,
        t)
    
    anoms = ad.get_timeseries(
    #     phenomenon_time_range=data_range,
        phenomenon_time_range=pt_range_z,
        num_time_slots=len(feature_time_slots),
        get_observations=get_observations_func,
        detector_method=detector_method,
        detector_params=detector_params,
        shift=shift,
        use_baseline=use_baseline,
        extend_range=extend_range,
    )
    
    anoms["feature_time_slots"] = feature_time_slots

    return anoms

In [5]:
def highlight(indices, alpha, color, ax):
    i=0
    while i<len(indices):
        ax.axvspan(indices[i]-0.5, indices[i]+0.5, facecolor=color, edgecolor='none', alpha=alpha)
        i+=1

In [6]:
colors = ['r', 'g', 'c', 'm', 'y', 'k']
results = []

def plot(detectors, hlt_detector):
    results = detectors
    fig, ax1 = plt.subplots(figsize=(20,7))
    first_result = detectors[list(detectors.keys())[0]]
    
    ts = pd.DataFrame({
        'values': [float(n) for n in first_result["property_values"]]
    }, index=[n.lower.strftime("%-d.%-m.%Y") for n in first_result["feature_time_slots"]])
    
    values_line = ts['values'].plot.line(ax=ax1, color='b')
    ax1.set_ylabel('values', color='b')
    ax1.tick_params('y', colors='b')
    
    lns = [values_line.get_lines()[0]]
    
    for i in range(len(detectors.keys())):
        detector = list(detectors.keys())[i]
        color = colors[i]
        anomalies = detectors[detector]["property_anomaly_rates"]

        ts = pd.DataFrame({
            'anomalies': anomalies
        })
        
        ax2 = ax1.twinx()
        anomalies_line = ts['anomalies'].plot.line(ax=ax2, color=color, label=detector)
        lns.append(anomalies_line.get_lines()[0])
        ax2.tick_params('y', colors=color)
        
        if detector == hlt_detector:
            sev = detectors[detector]["property_anomaly_severities"]
            for s in sev.keys():
                highlight(ts[ts['anomalies'] > sev[s]].index, s*0.006, color, ax2)
            

    labs = [ln.get_label() for ln in lns]
    ax1.legend(lns, labs, loc=1)
    
    # baseName = f"{baserange.lower.date()}..{baserange.upper.date()}"
    rangeName = f"{first_result['phenomenon_time_range'].lower.date()}..{first_result['phenomenon_time_range'].upper.date()}"
    # plt.savefig(f"graphs/{baseName}_{rangeName}_window-{str(window_size)}_prec-{str(detector_params['precision'])}.png", format="png")
    # plt.savefig(f"graphs/{rangeName}_window-{str(window_size)}_prec-{str(detector_params['precision'])}.png", format="png")

In [7]:
detectors = ["bitmap_mod", "bitmap_mod_shift", "bitmap_detector", "default_detector", "derivative_detector", "exp_avg_detector", "absolute_threshold", "diff_percent_threshold", "sign_test"]

In [8]:
def val_bitmap_mod(feature, start_date, end_date, detector_params):
    return detect_anomalies(start_date, end_date, "bitmap_mod", shift=False, feature=feature, detector_params=detector_params)
def val_bitmap_mod_shift(feature, start_date, end_date, detector_params):
    return detect_anomalies(start_date, end_date, "bitmap_mod_shift", feature=feature, detector_params=detector_params)
def val_bitmap_detector(feature, start_date, end_date, detector_params):
    return detect_anomalies(start_date, end_date, "bitmap_detector", feature=feature, detector_params=detector_params)
def val_default_detector(feature, start_date, end_date, detector_params):
    return detect_anomalies(start_date, end_date, "default_detector", feature=feature, detector_params={}, use_baseline=False, extend_range=False, shift=False)
def val_derivative_detector(feature, start_date, end_date, detector_params):
    return detect_anomalies(start_date, end_date, "derivative_detector", feature=feature, detector_params={}, use_baseline=False, extend_range=False, shift=False)
def val_exp_avg_detector(feature, start_date, end_date, detector_params):
    return detect_anomalies(start_date, end_date, "exp_avg_detector", feature=feature, detector_params={}, use_baseline=False, extend_range=False, shift=False)
def val_absolute_threshold(feature, start_date, end_date, detector_params):
    return detect_anomalies(t_from, t_to, "absolute_threshold", feature=feature, detector_params={}, use_baseline=False, extend_range=False, shift=False)
def val_diff_percent_threshold(feature, start_date, end_date, detector_params):
    return detect_anomalies(t_from, t_to, "diff_percent_threshold", feature=feature, detector_params={}, use_baseline=False, extend_range=False, shift=False)
def val_sign_test(feature, start_date, end_date, detector_params):
    return detect_anomalies(t_from, t_to, "sign_test", feature=feature, detector_params={}, use_baseline=False, extend_range=False, shift=False)

def plot_anomalies(feature, start_date, end_date, precision, window_size, chunk_size, hlt_detector, bitmap_mod, bitmap_mod_shift, bitmap_detector, default_detector, derivative_detector, exp_avg_detector):
    args = [feature, start_date, end_date, {
        "precision": precision,
        "lag_window_size": window_size,
        "future_window_size": window_size,
        "chunk_size": chunk_size
    }]
    anomalies = {}
    
    if bitmap_mod:
        anomalies["bitmap_mod"] = val_bitmap_mod(*args)
    if bitmap_mod_shift:
        anomalies["bitmap_mod_shift"] = val_bitmap_mod_shift(*args)
    if bitmap_detector:
        anomalies["bitmap_detector"] = val_bitmap_detector(*args)
    if default_detector:
        anomalies["default_detector"] = val_default_detector(*args)
    if derivative_detector:
        anomalies["derivative_detector"] = val_derivative_detector(*args)
    if exp_avg_detector:
        anomalies["exp_avg_detector"] = val_exp_avg_detector(*args)
            
#     if absolute_threshold:
#         anomalies["absolute_threshold"] = val_absolute_threshold(*args)
#     if diff_percent_threshold:
#         anomalies["diff_percent_threshold"] = val_diff_percent_threshold(*args)
#     if sign_test:
#         anomalies["sign_test"] = val_sign_test(*args)
        
        
    if len(anomalies.keys()) > 1:
        i = 0
        while hlt_detector not in anomalies.keys():
            hlt_detector = detectors[i]
            i += 1
        plot(anomalies, hlt_detector)


In [9]:
t_from = "2019-01-01"
t_to = "2019-03-30"
interact_manual(plot_anomalies,
                feature=features,
                start_date=widgets.DatePicker(value=pd.to_datetime(t_from).date(), description="Start date"),
                end_date=widgets.DatePicker(value=pd.to_datetime(t_to).date(), description="End date"),
                bitmap_mod=widgets.Checkbox(value=True,description="Bitmap mod"),
                bitmap_mod_shift=widgets.Checkbox(value=True,description="Bitmap mod shift"),
                bitmap_detector=widgets.Checkbox(value=True,description="LinkedIn bitmap"),
                default_detector=widgets.Checkbox(value=False,description="Default"),
                derivative_detector=widgets.Checkbox(value=False,description="Derivative"),
                exp_avg_detector=widgets.Checkbox(value=False,description="Exponential average"),
                #          absolute_threshold=widgets.Checkbox(value=False,description="Absolute threshold"),
                #          diff_percent_threshold=widgets.Checkbox(value=False,description="Diff Percent"),
                #          sign_test=widgets.Checkbox(value=False,description="Sign test"),
                precision=widgets.IntSlider(value=6, min=2, max=16, step=1, description="Precision"),
                window_size=widgets.BoundedIntText(value=96, min=16, max=256, step=1, description="Window size"),
                chunk_size=widgets.IntSlider(value=2, min=2, max=16, step=1, description="Chunk size"),
                hlt_detector=detectors,
        )

interactive(children=(Dropdown(description='feature', options=(<SamplingFeature: Brno, botanical garden PřF MU…

<function __main__.plot_anomalies(feature, start_date, end_date, precision, window_size, chunk_size, hlt_detector, bitmap_mod, bitmap_mod_shift, bitmap_detector, default_detector, derivative_detector, exp_avg_detector)>